# ZVP-based reverse-engineering

In [ ]:
import io
import numpy as np
from sympy import FF, sympify, symbols, Poly
import random
import tabulate
import pickle
from functools import partial
from itertools import product
from IPython.display import HTML, display
from tqdm.notebook import tqdm
from anytree import RenderTree, PreOrderIter
from concurrent.futures import ProcessPoolExecutor, as_completed

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters, load_params_ecgen
from pyecsca.ec.formula import FormulaAction, AdditionFormula, DoublingFormula
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod, gcd, SymbolicMod
from pyecsca.sca.re.tree import Map, Tree
from pyecsca.sca.re.rpa import MultipleContext
from pyecsca.sca.re.zvp import zvp_points, compute_factor_set
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mult import LTRMultiplier, AccumulationOrder
from pyecsca.misc.cfg import getconfig
from pyecsca.ec.error import NonInvertibleError, UnsatisfiedAssumptionError
from pyecsca.sca.re.zvp import unroll_formula, compute_factor_set, zvp_points, addition_chain, precomp_zvp_points

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)

## Exploration
First lets explore the behavior of addition formulas. The following two cells pick a coordinate model along with some formulas and symbolically unroll a scalar multiplication.

In [ ]:
which = "jacobian"
coords = model.coordinates[which]

In [ ]:
getconfig().ec.mod_implementation = "symbolic"
x, y, z = symbols("x y z")

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
field = FF(p)
a = SymbolicMod(Poly(0x4833d7aa73fa6694, x, y, z, domain=field), p)
b = SymbolicMod(Poly(0xa6c44a61c5323f6a, x, y, z, domain=field), p)
gx = SymbolicMod(Poly(0x5fd1f7d38d4f2333, x, y, z, domain=field), p)
gy = SymbolicMod(Poly(0x21f43957d7e20ceb, x, y, z, domain=field), p)
n = 0xc50de885003b80eb
h = 1

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)


add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
mult = LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqRP, True, True)


point = Point(coords,
              X=SymbolicMod(Poly(x, x, y, z, domain=field), params.curve.prime),
              Y=SymbolicMod(Poly(y, x, y, z, domain=field), params.curve.prime),
              Z=SymbolicMod(Poly(z, x, y, z, domain=field), params.curve.prime))
mult.init(params, point)
res = mult.multiply(5)

The result is a Point with coordinates that are polynomials in the input coordinates and curve parameters.

In [ ]:
cfg = getconfig()
cfg.ec.mod_implementation = "gmp"

## Reverse-engineering
Now, lets look at using the ZVP attack for reverse-engineering. First pick 10 random curves, some with $a \in \{0, -1, -3 \}$. The randomcurves are not special in any way and just serve to randomize the process, as the existence of ZVP points for a given intermediate value polynomial depends on the curve.

In [ ]:
curves = list(map(lambda spec: load_params_ecgen(io.BytesIO(spec.encode()), "affine"), [
    # Random
    """[{"field":{"p":"0xddf438409fc35161"},"a":"0x94d919b72f7dc6d8","b":"0x9f39032abb23f62a","order":"0xddf4383ffa8e6de7","subgroups":[{"x":"0xd5673b3fe176fc6b","y":"0x2d5b0a5bb2141317","order":"0xddf4383ffa8e6de7","cofactor":"0x1","points":[{"x":"0xd5673b3fe176fc6b","y":"0x2d5b0a5bb2141317","order":"0xddf4383ffa8e6de7"}]}]}]""",
    """[{"field":{"p":"0xa42c1467a1ed04f3"},"a":"0x55d07340a4572f2d","b":"0x0a938c37dfb0b6d5","order":"0xa42c14689284d3a7","subgroups":[{"x":"0x8633981c83ed43a2","y":"0x7b5374e9d7997199","order":"0xa42c14689284d3a7","cofactor":"0x1","points":[{"x":"0x8633981c83ed43a2","y":"0x7b5374e9d7997199","order":"0xa42c14689284d3a7"}]}]}]""",
    """[{"field":{"p":"0xea0d9cead19016ab"},"a":"0xcbbfe501c4ef6d92","b":"0x5762de777a6d9178","order":"0xea0d9cea8cd2c857","subgroups":[{"x":"0xe7daa3e061c3111b","y":"0x56ee59a6845c5e93","order":"0xea0d9cea8cd2c857","cofactor":"0x1","points":[{"x":"0xe7daa3e061c3111b","y":"0x56ee59a6845c5e93","order":"0xea0d9cea8cd2c857"}]}]}]""",
    """[{"field":{"p":"0x9c7e7216decb71a7"},"a":"0x324ef48887401a87","b":"0x3ce6f35a00280102","order":"0x9c7e72175ebfe709","subgroups":[{"x":"0x34683229b405418d","y":"0x308c923cae004514","order":"0x9c7e72175ebfe709","cofactor":"0x1","points":[{"x":"0x34683229b405418d","y":"0x308c923cae004514","order":"0x9c7e72175ebfe709"}]}]}]""",
    """[{"field":{"p":"0xeb5779f0bbf1ef5b"},"a":"0x2419e8bbc7b5f8f2","b":"0xe74e5d3064a4f2e3","order":"0xeb5779f21320c2e9","subgroups":[{"x":"0x3b6c269560abeb00","y":"0x29d157628e75e1c0","order":"0xeb5779f21320c2e9","cofactor":"0x1","points":[{"x":"0x3b6c269560abeb00","y":"0x29d157628e75e1c0","order":"0xeb5779f21320c2e9"}]}]}]""",
    """[{"field":{"p":"0x97b6ea097868b95d"},"a":"0x550a41d65e4bcd13","b":"0x47c5e527113b261c","order":"0x97b6ea094947a76b","subgroups":[{"x":"0x1e669fe19c865bd9","y":"0x05a6bb891920440f","order":"0x97b6ea094947a76b","cofactor":"0x1","points":[{"x":"0x1e669fe19c865bd9","y":"0x05a6bb891920440f","order":"0x97b6ea094947a76b"}]}]}]""",
    """[{"field":{"p":"0xa00629e6522032f7"},"a":"0x896f04a7ae302922","b":"0x6bc03365b1f1cb50","order":"0xa00629e5c03cf913","subgroups":[{"x":"0x14b7b48954936d4e","y":"0x670dc776273bf899","order":"0xa00629e5c03cf913","cofactor":"0x1","points":[{"x":"0x14b7b48954936d4e","y":"0x670dc776273bf899","order":"0xa00629e5c03cf913"}]}]}]""",
    """[{"field":{"p":"0xd47ec1d03a62686d"},"a":"0xd00a3ee0f5c86b02","b":"0x457a5b6c47db38d8","order":"0xd47ec1d107db7d6f","subgroups":[{"x":"0x41ebc3b763f3cd1b","y":"0x3d6925f214620e0c","order":"0xd47ec1d107db7d6f","cofactor":"0x1","points":[{"x":"0x41ebc3b763f3cd1b","y":"0x3d6925f214620e0c","order":"0xd47ec1d107db7d6f"}]}]}]""",
    """[{"field":{"p":"0xb1c9115c6f40d755"},"a":"0x79d3ceefafc44ce9","b":"0x8316af84264df42b","order":"0xb1c9115d17f84a45","subgroups":[{"x":"0x8b0a274089b53fe5","y":"0x3508d33c4beba5ad","order":"0xb1c9115d17f84a45","cofactor":"0x1","points":[{"x":"0x8b0a274089b53fe5","y":"0x3508d33c4beba5ad","order":"0xb1c9115d17f84a45"}]}]}]""",
    """[{"field":{"p":"0x8f738fda18cd5dff"},"a":"0x4747f2f9b8628cbf","b":"0x586cdb9378a1389f","order":"0x8f738fd8fc7ebed3","subgroups":[{"x":"0x7ad306c73b64c1b5","y":"0x69e3ca555190da4b","order":"0x8f738fd8fc7ebed3","cofactor":"0x1","points":[{"x":"0x7ad306c73b64c1b5","y":"0x69e3ca555190da4b","order":"0x8f738fd8fc7ebed3"}]}]}]""",
    # a = -1
    """[{"field":{"p":"0xcfef393139c3007f"},"a":"0xcfef393139c3007e","b":"0x950312812acb155f","order":"0xcfef39320179387b","subgroups":[{"x":"0xae2d2f58ca5b5cf7","y":"0xc3a4bf3a1dc10005","order":"0xcfef39320179387b","cofactor":"0x1","points":[{"x":"0xae2d2f58ca5b5cf7","y":"0xc3a4bf3a1dc10005","order":"0xcfef39320179387b"}]}]}]""",
    """[{"field":{"p":"0xb0461c0e4946cbd5"},"a":"0xb0461c0e4946cbd4","b":"0x082c3722016def51","order":"0xb0461c0e07e3e1bf","subgroups":[{"x":"0x5142200263be1fe3","y":"0x14984b7551ed21a9","order":"0xb0461c0e07e3e1bf","cofactor":"0x1","points":[{"x":"0x5142200263be1fe3","y":"0x14984b7551ed21a9","order":"0xb0461c0e07e3e1bf"}]}]}]""",
    """[{"field":{"p":"0xeff607c2dc4f278b"},"a":"0xeff607c2dc4f278a","b":"0x26fd03674f5092d2","order":"0xeff607c30ab8c50d","subgroups":[{"x":"0x004d4a5a9bb849fe","y":"0x80eb7ef89110c149","order":"0xeff607c30ab8c50d","cofactor":"0x1","points":[{"x":"0x004d4a5a9bb849fe","y":"0x80eb7ef89110c149","order":"0xeff607c30ab8c50d"}]}]}]""",
    """[{"field":{"p":"0xedc14fda51686379"},"a":"0xedc14fda51686378","b":"0xb3973a86901e3364","order":"0xedc14fda0cdbc199","subgroups":[{"x":"0xc76f0776feb59336","y":"0x625adaf0fb44ab9f","order":"0xedc14fda0cdbc199","cofactor":"0x1","points":[{"x":"0xc76f0776feb59336","y":"0x625adaf0fb44ab9f","order":"0xedc14fda0cdbc199"}]}]}]""",
    """[{"field":{"p":"0xfc6ee07288f1b78f"},"a":"0xfc6ee07288f1b78e","b":"0xe18821a83ca2ca30","order":"0xfc6ee0713e07f37f","subgroups":[{"x":"0x339d01a4b0db428e","y":"0x68100d42e5ffd979","order":"0xfc6ee0713e07f37f","cofactor":"0x1","points":[{"x":"0x339d01a4b0db428e","y":"0x68100d42e5ffd979","order":"0xfc6ee0713e07f37f"}]}]}]""",
    """[{"field":{"p":"0xa03c03a0072f69b3"},"a":"0xa03c03a0072f69b2","b":"0x3208ecebb633b82c","order":"0xa03c039ff31e37a7","subgroups":[{"x":"0x8134208d53e6f6c0","y":"0x6245db54032630a6","order":"0xa03c039ff31e37a7","cofactor":"0x1","points":[{"x":"0x8134208d53e6f6c0","y":"0x6245db54032630a6","order":"0xa03c039ff31e37a7"}]}]}]""",
    """[{"field":{"p":"0xbc8c6e7ce26746d9"},"a":"0xbc8c6e7ce26746d8","b":"0xb7e2b4fb2d769c4e","order":"0xbc8c6e7ba032dda7","subgroups":[{"x":"0x8e3c9cd771e7ffd8","y":"0x4dd02403ca890c5a","order":"0xbc8c6e7ba032dda7","cofactor":"0x1","points":[{"x":"0x8e3c9cd771e7ffd8","y":"0x4dd02403ca890c5a","order":"0xbc8c6e7ba032dda7"}]}]}]""",
    """[{"field":{"p":"0x9ccda4c062b95787"},"a":"0x9ccda4c062b95786","b":"0x31fcbb278951e3b9","order":"0x9ccda4bfae73e4f5","subgroups":[{"x":"0x303ac583c81644e3","y":"0x76713f6f470e94a0","order":"0x9ccda4bfae73e4f5","cofactor":"0x1","points":[{"x":"0x303ac583c81644e3","y":"0x76713f6f470e94a0","order":"0x9ccda4bfae73e4f5"}]}]}]""",
    """[{"field":{"p":"0xa339e3745518416f"},"a":"0xa339e3745518416e","b":"0x52d39a67f2401673","order":"0xa339e3743950389b","subgroups":[{"x":"0x6b8986f706afac58","y":"0x5c901b1afa0b64da","order":"0xa339e3743950389b","cofactor":"0x1","points":[{"x":"0x6b8986f706afac58","y":"0x5c901b1afa0b64da","order":"0xa339e3743950389b"}]}]}]""",
    """[{"field":{"p":"0x8b7d2baee4e47311"},"a":"0x8b7d2baee4e47310","b":"0x21ab23afb5a9e2ca","order":"0x8b7d2baf201f2bdd","subgroups":[{"x":"0x797c1dec0d73ec64","y":"0x28f90926ea9c6b33","order":"0x8b7d2baf201f2bdd","cofactor":"0x1","points":[{"x":"0x797c1dec0d73ec64","y":"0x28f90926ea9c6b33","order":"0x8b7d2baf201f2bdd"}]}]}]""",
    # a = -3
    """[{"field":{"p":"0x8d79ca36cee026a7"},"a":"0x8d79ca36cee026a4","b":"0x0478c1f80ce2c9c6","order":"0x8d79ca35a428c76f","subgroups":[{"x":"0x2e94a3e38f8b345e","y":"0x83e6c6f0cb8f69c4","order":"0x8d79ca35a428c76f","cofactor":"0x1","points":[{"x":"0x2e94a3e38f8b345e","y":"0x83e6c6f0cb8f69c4","order":"0x8d79ca35a428c76f"}]}]}]""",
    """[{"field":{"p":"0x48e1a125250323a7"},"a":"0x48e1a125250323a4","b":"0x02a4d99f41d23210","order":"0x48e1a124f895db6d","subgroups":[{"x":"0x409e15d65fcae55a","y":"0x207e142056d62d07","order":"0x48e1a124f895db6d","cofactor":"0x1","points":[{"x":"0x409e15d65fcae55a","y":"0x207e142056d62d07","order":"0x48e1a124f895db6d"}]}]}]""",
    """[{"field":{"p":"0xcb5aa8a7a10aa06b"},"a":"0xcb5aa8a7a10aa068","b":"0x31fe9c57c570174f","order":"0xcb5aa8a6cf812191","subgroups":[{"x":"0x84c75d46fc687ff1","y":"0x7424362ac73df187","order":"0xcb5aa8a6cf812191","cofactor":"0x1","points":[{"x":"0x84c75d46fc687ff1","y":"0x7424362ac73df187","order":"0xcb5aa8a6cf812191"}]}]}]""",
    """[{"field":{"p":"0xba965ca9c8aa0a1b"},"a":"0xba965ca9c8aa0a18","b":"0x676535a1eaf5c605","order":"0xba965caae5741b6f","subgroups":[{"x":"0x313d58c47b8ed95f","y":"0x991ba98cbbb0fe9f","order":"0xba965caae5741b6f","cofactor":"0x1","points":[{"x":"0x313d58c47b8ed95f","y":"0x991ba98cbbb0fe9f","order":"0xba965caae5741b6f"}]}]}]""",
    """[{"field":{"p":"0xbfb7747454a17d15"},"a":"0xbfb7747454a17d12","b":"0x611b69881db4ce69","order":"0xbfb7747547fd57d3","subgroups":[{"x":"0x3385044d698640fc","y":"0x50cee623251b559e","order":"0xbfb7747547fd57d3","cofactor":"0x1","points":[{"x":"0x3385044d698640fc","y":"0x50cee623251b559e","order":"0xbfb7747547fd57d3"}]}]}]""",
    """[{"field":{"p":"0x99235f1ed44b3959"},"a":"0x99235f1ed44b3956","b":"0x5d8dda19dbe804d4","order":"0x99235f1d975f376d","subgroups":[{"x":"0x4fed262974c1d800","y":"0x27590c454edd59ca","order":"0x99235f1d975f376d","cofactor":"0x1","points":[{"x":"0x4fed262974c1d800","y":"0x27590c454edd59ca","order":"0x99235f1d975f376d"}]}]}]""",
    """[{"field":{"p":"0xa7ff74a0dc8c161d"},"a":"0xa7ff74a0dc8c161a","b":"0x583b968bb611b284","order":"0xa7ff74a06811ee75","subgroups":[{"x":"0x5f5c76454edf12e7","y":"0x4c73cbfc44f41508","order":"0xa7ff74a06811ee75","cofactor":"0x1","points":[{"x":"0x5f5c76454edf12e7","y":"0x4c73cbfc44f41508","order":"0xa7ff74a06811ee75"}]}]}]""",
    """[{"field":{"p":"0xb52c62ca8703a063"},"a":"0xb52c62ca8703a060","b":"0x0baec43a07b54c21","order":"0xb52c62c963037121","subgroups":[{"x":"0x6fe4a521a29bc1ab","y":"0x3fca7180021f8f0f","order":"0xb52c62c963037121","cofactor":"0x1","points":[{"x":"0x6fe4a521a29bc1ab","y":"0x3fca7180021f8f0f","order":"0xb52c62c963037121"}]}]}]""",
    """[{"field":{"p":"0xb8921f25b6ce5267"},"a":"0xb8921f25b6ce5264","b":"0xa575c9f97563265d","order":"0xb8921f2592b6b39f","subgroups":[{"x":"0x7eb120fada47765c","y":"0x64ef4e51d4159304","order":"0xb8921f2592b6b39f","cofactor":"0x1","points":[{"x":"0x7eb120fada47765c","y":"0x64ef4e51d4159304","order":"0xb8921f2592b6b39f"}]}]}]""",
    """[{"field":{"p":"0xc591b8c4df0afc19"},"a":"0xc591b8c4df0afc16","b":"0x0a1eb46a6e647f0a","order":"0xc591b8c3eb07239f","subgroups":[{"x":"0x1963bfb862cb0bf3","y":"0x30da8bb7fa77277d","order":"0xc591b8c3eb07239f","cofactor":"0x1","points":[{"x":"0x1963bfb862cb0bf3","y":"0x30da8bb7fa77277d","order":"0xc591b8c3eb07239f"}]}]}]""",
    # a = 0
    """[{"field":{"p":"0xceaf446a53f14bc1"},"a":"0x0000000000000000","b":"0x326539376260f173","order":"0xceaf446aae275419","subgroups":[{"x":"0x98fe44948c3f8678","y":"0x3d440ee959a912d7","order":"0xceaf446aae275419","cofactor":"0x1","points":[{"x":"0x98fe44948c3f8678","y":"0x3d440ee959a912d7","order":"0xceaf446aae275419"}]}]}]""",
    """[{"field":{"p":"0xb3c2beca75d66de3"},"a":"0x0000000000000000","b":"0x46069225826b51aa","order":"0xb3c2bec95881b695","subgroups":[{"x":"0x81500c226efa0d5a","y":"0x674e09d296452eee","order":"0xb3c2bec95881b695","cofactor":"0x1","points":[{"x":"0x81500c226efa0d5a","y":"0x674e09d296452eee","order":"0xb3c2bec95881b695"}]}]}]""",
    """[{"field":{"p":"0xd6097c1ce207aae7"},"a":"0x0000000000000000","b":"0x7adaab54e7dfd564","order":"0xd6097c1b407eb413","subgroups":[{"x":"0x151da8fb1f83201e","y":"0x8bfeb90ec1177a91","order":"0xd6097c1b407eb413","cofactor":"0x1","points":[{"x":"0x151da8fb1f83201e","y":"0x8bfeb90ec1177a91","order":"0xd6097c1b407eb413"}]}]}]""",
    """[{"field":{"p":"0x97a3e2d617a2309d"},"a":"0x0000000000000000","b":"0x7f311cba46652247","order":"0x97a3e2d712ffd715","subgroups":[{"x":"0x46d725812af15870","y":"0x727f88365dbd0e80","order":"0x97a3e2d712ffd715","cofactor":"0x1","points":[{"x":"0x46d725812af15870","y":"0x727f88365dbd0e80","order":"0x97a3e2d712ffd715"}]}]}]""",
    """[{"field":{"p":"0xd43c9a4cd686a599"},"a":"0x0000000000000000","b":"0xc795a256566ee407","order":"0xd43c9a4cb750f1a5","subgroups":[{"x":"0x4f1fcb572419ce5c","y":"0x5f54d643e16732e4","order":"0xd43c9a4cb750f1a5","cofactor":"0x1","points":[{"x":"0x4f1fcb572419ce5c","y":"0x5f54d643e16732e4","order":"0xd43c9a4cb750f1a5"}]}]}]""",
    """[{"field":{"p":"0x847de883ab4fbf4d"},"a":"0x0000000000000000","b":"0x09866366b3b45c2d","order":"0x847de8837e6d4477","subgroups":[{"x":"0x62fd7b4bc7c9acb4","y":"0x2d0942774607106b","order":"0x847de8837e6d4477","cofactor":"0x1","points":[{"x":"0x62fd7b4bc7c9acb4","y":"0x2d0942774607106b","order":"0x847de8837e6d4477"}]}]}]""",
    """[{"field":{"p":"0xf0d617c3c47b7c77"},"a":"0x0000000000000000","b":"0xd856b3dcb95764a2","order":"0xf0d617c5512cec85","subgroups":[{"x":"0xeaf9b352a3daac45","y":"0x4e4e557f9fc3febc","order":"0xf0d617c5512cec85","cofactor":"0x1","points":[{"x":"0xeaf9b352a3daac45","y":"0x4e4e557f9fc3febc","order":"0xf0d617c5512cec85"}]}]}]""",
    """[{"field":{"p":"0xce920b656c80b373"},"a":"0x0000000000000000","b":"0xb4a07dfae71ddc62","order":"0xce920b65eee38015","subgroups":[{"x":"0x7895c02b3c5205b5","y":"0x2926be6446b98d62","order":"0xce920b65eee38015","cofactor":"0x1","points":[{"x":"0x7895c02b3c5205b5","y":"0x2926be6446b98d62","order":"0xce920b65eee38015"}]}]}]""",
    """[{"field":{"p":"0xb6d4e25f76cc9df7"},"a":"0x0000000000000000","b":"0x18e95a2283692623","order":"0xb6d4e25ea270ed03","subgroups":[{"x":"0x2da7a97d5d899bc5","y":"0x17d27fd34562e3d9","order":"0xb6d4e25ea270ed03","cofactor":"0x1","points":[{"x":"0x2da7a97d5d899bc5","y":"0x17d27fd34562e3d9","order":"0xb6d4e25ea270ed03"}]}]}]""",
    """[{"field":{"p":"0xe7cd1979ebed69ed"},"a":"0x0000000000000000","b":"0x278e92b83191a7da","order":"0xe7cd197966893365","subgroups":[{"x":"0xc4de44402da5b9a6","y":"0x2b45e7f32e3701ba","order":"0xe7cd197966893365","cofactor":"0x1","points":[{"x":"0xc4de44402da5b9a6","y":"0x2b45e7f32e3701ba","order":"0xe7cd197966893365"}]}]}]"""
    # b = 0 (causes more issues than gain) but the (w12-0) coords actually require it...
    #"""[{"field":{"p":"0x9d9119957f02fe3f"},"a":"0x0106903196d88df9","b":"0x0000000000000000","order":"0x9d9119957f02fe40","subgroups":[{"x":"0x191a36b9cd81de96","y":"0x10f2c6bded391aa9","order":"0x9d9119957f02fe40","cofactor":"0x1","points":[{"x":"0x0000000000000000","y":"0x0000000000000000","order":"0x2"},{"x":"0x95913fae9065da0f","y":"0x5eeddeee7152d6fb","order":"0x276446655fc0bf9"}]}]}]"""
]))

for i, params in enumerate(curves):
    curve = params.curve
    if curve.parameters["a"] == -3:
        params.name = "a=-3"
    elif curve.parameters["a"] == -1:
        params.name = "a=-1"
    elif curve.parameters["a"] == 0:
        params.name = "a=0"
    else:
        params.name = "random"
    params.name += f"[{i}]"

First lets fix some scalars, go over the curves and compute the addition chain to obtain information about which multiples of the input point will go into the formulas.

In [ ]:
scalars = [0b1111110, 0b1011101, 0b110110, 0b100100, 0b1000110, 0b1001101, 0b101001, 0b1100100, 0b1010110, 0b101010] * 4

chains = []
scalar_map = {}
ops = set()
for scalar, params in zip(scalars, curves):
    chain = addition_chain(scalar, params, LTRMultiplier, lambda add,dbl: LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqRP, True, True))
    chains.append(chain)
    scalar_map[params] = scalar
    ops.update(chain)
print(sorted(list(ops)))

Now, lets compute the sets of ZVP points, going over all coordinate systems and all of their formulas (that fit the scalar multiplier) and store them into the `point_chains`. These items form individual distinct entries of the distinguishing table.

In [ ]:
formula_classes = [AdditionFormula, DoublingFormula]
factor_sets = {}
for coord_name, coords in model.coordinates.items():
    formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]

    for formula_group, formula_class in zip(formula_groups, formula_classes):
        for formula in formula_group:
            fset = compute_factor_set(formula, filter_nonhomo=False)
            if formula_class == DoublingFormula:
                new_fset = set()
                for poly in fset:
                    pl = poly.copy()
                    for symbol in poly.free_symbols:
                        original = str(symbol)
                        if original.endswith("1"):
                            new = original.replace("1", "2")
                            pl = pl.subs(original, new)
                    new_fset.add(pl)
                fset = new_fset
            factor_sets[formula] = fset

polynomials = {}
for coord_name, coords in model.coordinates.items():
    for chain, affine_params in zip(chains, curves):
        try:
            params = affine_params.to_coords(coords)
        except UnsatisfiedAssumptionError:
            continue
        add_polynomials = set()
        dbl_polynomials = set()
        formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]
        for formula_group, formula_class in zip(formula_groups, formula_classes):
            for formula in formula_group:
                if formula_class == AdditionFormula:
                    add_polynomials.update(factor_sets[formula])
                else:
                    dbl_polynomials.update(factor_sets[formula])
        polynomials[params] = {
            "add": add_polynomials,
            "dbl": dbl_polynomials
        }

In [ ]:
# bound is the maximal dlog in the hard case of the DCP to be solved
bound = 100

all_points = set()
with ProcessPoolExecutor(max_workers=10) as pool:
    futures = []
    args = []
    for coord_name, coords in model.coordinates.items():
        for chain, affine_params in zip(chains, curves):
            try:
                params = affine_params.to_coords(coords)
            except UnsatisfiedAssumptionError:
                continue
            for op, ks in chain:
                if len(ks) == 1:
                    k = ks[0]
                else:
                    # zvp_points assumes (P, [k]P)
                    ks_mod = list(map(lambda v: Mod(v, params.order), ks))
                    k = int(ks_mod[1] / ks_mod[0])
                polys = polynomials[params][op]
                for poly in polys:
                    only_1 = all((not str(gen).endswith("2")) for gen in poly.gens)
                    only_2 = all((not str(gen).endswith("1")) for gen in poly.gens)
                    # This is the hard case where a dlog needs to be substituted, so bound it.
                    if not (only_1 or only_2) and k > bound:
                        continue
                    futures.append(pool.submit(zvp_points, poly, params.curve, k, params.order))
                    args.append((poly, affine_params, k))
    for future in tqdm(as_completed(futures), desc="Computing", total=len(futures), smoothing=0):
        j = futures.index(future)
        poly, affine_params, k = args[j]
        result = future.result()
        for point in result:
            all_points.add((point, affine_params))
print(f"Got {len(all_points)} points")

Our ZVP points might (due to the bounds above thus the incompleteness of our analysis) lead to more zeros than we attribute to them (in more configurations), i.e. "false negatives". They might also be erroneous and not lead to zeros if the argument `filter_nonhomo` is False, as non-homogenous intermediate polynomials are not filtered out of the analysis. They introduce "false positives" but also some true positives.

Thus we perform a remapping step where we execute the scalar multiplication with given points and trace whether they introduce the zeros. This gives us a new distinguishing map `remapped_hit_point_map`, now without "false negatives" or "false positives".

Note that we also construct two additional maps `remapped_count_point_map` and `remapped_position_point_map` which represent the results of remapping using an oracle counting the zeros and an oracle giving the positions of the zeros in the computation, respectively.

In [ ]:
def remap(coords, formulas, points, scalar_map):
    mult = LTRMultiplier(*formulas, None, False, AccumulationOrder.PeqRP, True, True)
    hit_points = set()
    count_points = {}
    position_points = {}
    
    param_map = {}
    for point, params in points:
        if params not in param_map:
            try:
                param_map[params] = params.to_coords(coords)
            except UnsatisfiedAssumptionError:
                param_map[params] = None
                continue
        elif param_map[params] is None:
            continue
        mult.init(param_map[params], point.to_model(param_map[params].curve.coordinate_model, param_map[params].curve))
        scalar = scalar_map[params]
        with local(DefaultContext()) as ctx:
            try:
                mult.multiply(scalar)
            except UnsatisfiedAssumptionError:
                continue
        trace = []
        def callback(action):
            if isinstance(action, FormulaAction):
                for intermediate in action.op_results:
                    trace.append(intermediate.value)
        ctx.actions.walk(callback)
        zeros = tuple(map(lambda x: int(x) == 0, trace))
        if any(zeros):
            hit_points.add((point, params))
        count_points[(point, params)] = sum(zeros)
        position_points[(point, params)] = zeros
    return hit_points, count_points, position_points

remapped_hit_point_map = {}
remapped_count_point_map = {}
remapped_position_point_map = {}
with ProcessPoolExecutor(max_workers=30) as pool:
    futures = []
    pairs = []
    for coord_name, coords in model.coordinates.items():
        formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]
        formula_combinations = list(product(*formula_groups))
        for formulas in formula_combinations:
            futures.append(pool.submit(remap, coords, formulas, all_points, scalar_map))
            pairs.append(formulas)
    for future in tqdm(as_completed(futures), total=len(futures), desc="Remapping", smoothing=0):
        j = futures.index(future)
        cfg = pairs[j]
        h, c, p = future.result()
        remapped_hit_point_map[cfg] = h
        remapped_count_point_map[cfg] = c
        remapped_position_point_map[cfg] = p

Finally, we can build a tree using the remapped distinguishing map.

In [ ]:
dmap_remapped = Map.from_binary_sets(set(remapped_hit_point_map.keys()), remapped_hit_point_map)

In [ ]:
tree_remapped = Tree.build(set(remapped_hit_point_map.keys()), dmap_remapped)

In [ ]:
print(tree_remapped.describe())

We can also investigate the other oracles and the distinguishing trees they can build:

In [ ]:
dmap_count = Map.from_io_map(set(remapped_count_point_map.keys()), remapped_count_point_map)

In [ ]:
tree_count = Tree.build(set(remapped_count_point_map.keys()), dmap_count)

In [ ]:
dmap_position = Map.from_io_map(set(remapped_position_point_map.keys()), remapped_position_point_map)

In [ ]:
tree_position = Tree.build(set(remapped_position_point_map.keys()), dmap_position)

In [ ]:
print("Zero hit")
print(tree_remapped.describe())
print("\nZero count")
print(tree_count.describe())
print("\nZero position")
print(tree_position.describe())

Now, lets examine the representation of factor sets and the distinguishing trees they can build.

In [ ]:
fset_map = {}
fset_nonhomo_map = {}
factor_sets = {}
factor_sets_nonhomo = {}
for coord_name, coords in model.coordinates.items():
    formula_groups = [list(filter(lambda formula: isinstance(formula, formula_class) and (formula.name.startswith("add") or formula.name.startswith("dbl")), coords.formulas.values())) for formula_class in formula_classes]
    for formula_group in formula_groups:
        for formula in formula_group:
            factor_sets[formula] = compute_factor_set(formula)
            factor_sets_nonhomo[formula] = compute_factor_set(formula, filter_nonhomo=False)
    formula_combinations = list(product(*formula_groups))
    for formulas in formula_combinations:
        fset = set()
        fset_nonhomo = set()
        for formula in formulas:
            fset.update(factor_sets[formula])
            fset_nonhomo.update(factor_sets_nonhomo[formula])
        fset_map[formulas] = fset
        fset_nonhomo_map[formulas] = fset_nonhomo

In [ ]:
dmap_fset = Map.from_binary_sets(set(fset_map.keys()), fset_map)
dmap_fset_nonhomo = Map.from_binary_sets(set(fset_nonhomo_map.keys()), fset_nonhomo_map)

In [ ]:
tree_fset = Tree.build(set(fset_map.keys()), dmap_fset)
tree_fset_nonhomo = Tree.build(set(fset_nonhomo_map.keys()), dmap_fset_nonhomo)

In [ ]:
print("Factor sets")
print(tree_fset.describe())
print("\nFactor sets (unfiltered)")
print(tree_fset_nonhomo.describe())

In [ ]:
expanded = tree_remapped.expand(dmap_fset)

In [ ]:
print("Zero hit + factor sets")
print(expanded.describe())

In [ ]:
p = set()
for node in PreOrderIter(expanded.root):
    if isinstance(node.dmap_input, Poly):
        print(node.dmap_input, [len(child.cfgs) for child in node.children])
        p.add(node.dmap_input)
print("---")
for pp in p:
    print(pp)
    for formula, fset in factor_sets_nonhomo.items():
        if pp in fset:
            print(formula)